<a href="https://colab.research.google.com/github/KurniaKhaikal/Cifar10k_TransferLearning/blob/main/Feature2_Cifar10_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob
import random
import matplotlib.pylab as plt

In [2]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"khaikal","key":"9e492a7553b8e713debda6f4f6810151"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d oxcdcd/cifar10

 93% 163M/175M [00:01<00:00, 112MB/s]
100% 175M/175M [00:01<00:00, 117MB/s]


In [4]:
import os
import zipfile

local_zip = '/content/cifar10.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/')
zip_ref.close()

In [5]:
!mkdir "/content/cifar10/Base2"

In [6]:
base_dataset = "/content/cifar10/Base2"

In [7]:
class_dir = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
IMAGE_SIZE = (224, 224)

In [8]:
!pip install split-folders

In [9]:
import splitfolders


splitfolders.ratio("/content/cifar10/test", output="/content/cifar10/Base2",
    seed=1337, ratio=(.9, .1), group_prefix=None, move=False)

Copying files: 10000 files [00:02, 3697.61 files/s]


In [10]:
import os
# indexing file images
dataset_train = []
for class_item in class_dir:
    cur_dir = os.path.join(base_dataset, 'train', class_item)
    for file in os.listdir(cur_dir):
        dataset_train.append(os.path.join(cur_dir, file))

In [11]:
print("train:", len(dataset_train))

train: 9000


In [25]:
dataset_train

['/content/cifar10/Base2/train/airplane/7594_airplane.png',
 '/content/cifar10/Base2/train/airplane/9356_airplane.png',
 '/content/cifar10/Base2/train/airplane/6653_airplane.png',
 '/content/cifar10/Base2/train/airplane/1409_airplane.png',
 '/content/cifar10/Base2/train/airplane/4270_airplane.png',
 '/content/cifar10/Base2/train/airplane/4802_airplane.png',
 '/content/cifar10/Base2/train/airplane/9888_airplane.png',
 '/content/cifar10/Base2/train/airplane/7871_airplane.png',
 '/content/cifar10/Base2/train/airplane/8862_airplane.png',
 '/content/cifar10/Base2/train/airplane/5302_airplane.png',
 '/content/cifar10/Base2/train/airplane/5587_airplane.png',
 '/content/cifar10/Base2/train/airplane/6549_airplane.png',
 '/content/cifar10/Base2/train/airplane/3601_airplane.png',
 '/content/cifar10/Base2/train/airplane/9182_airplane.png',
 '/content/cifar10/Base2/train/airplane/9980_airplane.png',
 '/content/cifar10/Base2/train/airplane/6476_airplane.png',
 '/content/cifar10/Base2/train/airplane/

In [12]:
from imutils import paths
import cv2
from sklearn.preprocessing import LabelBinarizer

In [14]:
print("[INFO] load images Cifar10k dataset...")
#  load images
train_images = []
for image_path in dataset_train:
    if ".png" or ".jpeg" in image_path:
        image = cv2.imread(image_path)
        image = cv2.resize(image, IMAGE_SIZE)
        train_images.append(image)

[INFO] load images Cifar10k dataset...


In [15]:
import numpy as np
# normalization
print("[INFO] normalization...")
train_x = np.array(train_images).astype("float32") / 255.0

[INFO] normalization...


In [16]:
import numpy as np
import json
import os
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model

In [20]:
!pip install keras

In [29]:
from tensorflow.keras.applications import ResNet50
modelRestNet = ResNet50(weights="imagenet")
model = Model(inputs=modelRestNet.input, outputs=modelRestNet.output)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [24]:
features = model.predict(train_x)

In [27]:
indexes = list(range(0, train_x.shape[0]))
features_array = [[float(x) for x in y] for y in features]
labels = [path.split("/")[5] for path in dataset_train]
data_train = {"indexes": indexes, "features": features_array, "locations": dataset_train, "labels":labels}

In [28]:
with open('/content/ResNet_feature_extraction.json', 'w') as f:
    json.dump(data_train, f)